# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [7]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import json
import pickle

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core.model import Model


from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

import urllib.request


import joblib


SDK version: 1.39.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# Choose a name for the run history container in the workspace.
experiment_name = 'hd-experiment-1'
project_folder = './hd-project'

experiment = Experiment(ws, experiment_name)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-191197
aml-quickstarts-191197
southcentralus
6971f5ac-8af1-446e-8034-05acea24681f


In [3]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [27]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Titanic_Dataset"
description_text = "Kaggle Titanic Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/camille-wilkens/AZ_Capstone/main/Titanic-Dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [28]:
dataset.take(5).to_pandas_dataframe()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [35]:

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling(
    {"--C":choice(0.5,1.0),     
    "--max_iter":choice(50,100,150)})  

# TODO: Create an early termination policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)


# Setup environment for your training run
#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-migrate-from-estimators-to-scriptrunconfig
#https://knowledge.udacity.com/questions/423888
#env_name = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#TODO: Create estimator 
est = SKLearn(source_directory='.', entry_script='train.py', compute_target=amlcompute_cluster_name)

#TODO: Create hyperdrive config
hyperdrive_config = HyperDriveConfig (
        hyperparameter_sampling=ps,
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        primary_metric_name='accuracy',
        policy=policy,
        max_total_runs=8, 
        max_concurrent_runs=4,
        estimator=est)


In [36]:
#TODO: Submit your experiment
run=experiment.submit(config=hyperdrive_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_97c5db2f-54e9-47e0-b64b-6df88fd345b5
Web View: https://ml.azure.com/runs/HD_97c5db2f-54e9-47e0-b64b-6df88fd345b5?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-191197/workspaces/quick-starts-ws-191197&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-04-06T21:32:41.694912][API][INFO]Experiment created<END>\n""<START>[2022-04-06T21:32:42.476471][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2022-04-06T21:32:43.2263101Z][SCHEDULER][INFO]Scheduling job, id='HD_97c5db2f-54e9-47e0-b64b-6df88fd345b5_0'<END><START>[2022-04-06T21:32:43.3843120Z][SCHEDULER][INFO]Scheduling job, id='HD_97c5db2f-54e9-47e0-b64b-6df88fd345b5_1'<END><START>[2022-04-06T21:32:43.5207884Z][SCHEDULER][INFO]Scheduling job, id='HD_97c5db2f-54e9-47e0-b64b-6df88fd345b5_2'<END>"<START>[2022-04-06T21:32:43.597058][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to 

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [23]:
### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 

print("Best Run:",best_run.id)
print("\n accuracy:", best_run_metrics['Accuracy'])
print("\n file name :",best_run.get_file_names())

if "outputs" not in os.listdir():
    os.mkdir("./outputs")

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [15]:
best_run.get_file_names()
best_run_metrics

{'Regularization Strength:': 1.0,
 'Max iterations:': 100,
 'Accuracy': 0.9094081942336874}

In [16]:
#TODO: Save the best model
#best_model = best_run.register_model(model_name='hyperdrive_model', tags=best_run_metrics,model_path='outputs/model.joblib')
best_model = best_run.register_model(model_name='hyperdrive_model',model_path='.')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

